# shop results

## goal

* find factors influencing this shop's results
* predict results

In [ ]:
from itertools import product

import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%matplotlib inline

## Load data

In [ ]:
from datetime import datetime, timedelta, time
from os import path, scandir

daily_datadir = "./data/CaisseJour/"
datadirs = [path.join(daily_datadir, d.name) for d in scandir(daily_datadir)]
data_files = [
    path.join(datadir, file.name) for datadir in datadirs for file in scandir(datadir)
]

In [ ]:
def parse_caisse(filename, keywords=("Chiffre", "TVA", "nombre", "moyen", "ticket")):
    """Parse file "caisse jour"
    
    Args:
        filename (string): file to parse
        keywords (list): list of keywords for  one line data
    """
    data = {}
    with open(filename, "br") as fd:
        for line in fd:
            line = line.decode("Windows-1252", errors="ignore")
            if "à" in line:
                # try with date
                try:
                    date = [int(d) for d in line.split(" ")[0].split("/")]
                except ValueError:
                    # "à" in cocktail name
                    continue
                data["date"] = datetime(date[2], date[1], date[0])
            elif any(keyword in line for keyword in keywords):
                value = line.split(";")[1]
                value = value.strip("€ \r\n")
                try:
                    # parse french number representation
                    value = value.replace(",", ".")
                    value = float(value)
                except ValueError:
                    # not a number, cannot convertto float
                    pass
                data[line.split(";")[0].strip()] = value
        # TODO: add small tables
    return data

In [ ]:
daily = pd.DataFrame(parse_caisse(f) for f in data_files)

In [ ]:
daily.profile_report()

In [ ]:
daily.index = daily["date"]  # keep date and index
open_days = daily["date"]

## Calendar

In [ ]:
start_date = min(daily["date"])
end_date = max(daily["date"])

# start_date = datetime(2018, 09, 01)
# end_date = datetime(2019, 09, 01)

In [ ]:
def date_to_monthweek(date):
    """Return the week number of the month, i.e. the number of mondays before this date.
    
    Args:
        date: (datetime.datetime)
        
    Return:
        int: the week number
    """
    return len(
        [
            day
            for day in pd.date_range(datetime(date.year, date.month, 1), date)
            if day.weekday() == 0
        ]
    )

In [ ]:
calendar = pd.DataFrame(pd.date_range(start_date, end_date), columns=("date",))
# calendar["day", "month", "year", "wod"] = list(map(lambda x: (x.day, x.month, x.year, x.weekday()), calendar["date"]))
calendar["day"] = list(map(lambda x: x.day, calendar["date"]))
calendar["month"] = list(map(lambda x: x.month, calendar["date"]))
calendar["year"] = list(map(lambda x: x.year, calendar["date"]))
calendar["dow"] = list(map(lambda x: x.weekday(), calendar["date"]))
calendar["week number"] = list(map(lambda x: x.isocalendar()[1], calendar["date"]))
calendar["month week number"] = list(map(date_to_monthweek, calendar["date"]))

### Holidays

from https://date.nager.at/PublicHoliday/DownloadCSV/FR/2018

In [ ]:
datadir = "./data/calendars"
data_files = [path.join(datadir, file.name) for file in scandir(datadir)]

holidays = pd.concat([pd.read_csv(file) for file in data_files])

# reformat date
holidays["Date"] = pd.Series(
    [
        datetime(int(x.split("-")[0]), int(x.split("-")[1]), int(x.split("-")[2]))
        for x in holidays["Date"]
    ]
)

In [ ]:
holidays.head()

In [ ]:
calendar["public holidays"] = list(
    map(lambda x: x in list(holidays["Date"]), calendar["date"])
)

In [ ]:
calendar = calendar.set_index("date")

In [ ]:
calendar.head()

join data

In [ ]:
# the pandas way
daily = daily.join([calendar])

# the spark.sql way
# TODO

daily.head()

## Weather

from meteofrance

### Programation

In [ ]:
import icalendar

In [ ]:
icalfile = "./data/prog/Programmation.ics"

columns = ("DTSTAMP", "UID", "SUMMARY", "DTSTART", "DTEND")

In [ ]:
with open(icalfile, "rb") as fd:
    icalcontent = fd.read()

events_cal = icalendar.Calendar().from_ical(icalcontent)

In [ ]:
events = pd.DataFrame()
for evt in events_cal.walk("vevent"):

    # rebuild event
    elt = {}
    for k, v in dict(evt).items():
        elt[k] = evt[k].to_ical().decode()
    events = events.append(pd.DataFrame([elt]), sort=False)

events.head()

In [ ]:
# parse date
date_cols = [col for col in events.columns if col.startswith("DT")]

for col in date_cols:
    events[col] = events[col].apply(pd.to_datetime)

events.head()

duration are not all filled

In [ ]:
events["DURATION"].unique()

In [ ]:
events[~events["DURATION"].isnull()].head()

In [ ]:
events[events["DTEND"].isnull() & events["DURATION"].isnull()]

either duration or end is filled.

In [ ]:
def compute_dtend(row):
    """Return DTEND value
    """
    if row["DTEND"] not in (pd.NaT, np.NaN):
        return row["DTEND"]
    dur = row["DURATION"].strip("PTM")
    parsed = dur.split("H")
    hour = parsed[0]
    minutes = parsed[1] if parsed[1] else "0"
    return row["DTSTART"] + timedelta(int(hour), int(minutes))

In [ ]:
events["DTEND"] = events.apply(compute_dtend, axis=1)
events[events["DTEND"].isnull()]

In [ ]:
events["DURATION"] = events["DTEND"] - events["DTSTART"]
events.head(10)

In [ ]:
events["DATE"] = events["DTSTART"].apply(lambda x: x.date())
events["START_TIME"] = events["DTSTART"].apply(lambda x: x.time())

In [ ]:
import csv

with open("./data/prog/type_spectacle.csv", "r") as fd:
    reader = csv.DictReader(fd)
    type_spectacle = {
        row["nom troupe"]: row["type spectacle"].lower() for row in reader
    }

In [ ]:
events["TYPE"] = events["SUMMARY"].apply(lambda x: type_spectacle[x])
events.head()

In [ ]:
events["TYPE"].unique()

## Data exploration

Describe and restrict features

In [ ]:
from pandas.plotting import scatter_matrix

In [ ]:
# data description
columns_descr = {
    "Chiffre d'Affaires HT": "(float) Income (taxes excluded)",
    "Chiffre d'Affaires TTC": "(float) Income (taxes included)",
    "Nombre moyen de produits / Ticket": "(float) Mean good numbers per transaction",
    "Ticket moyen TTC": "(float) Mean transaction value",
    "date": "(date) date of the day",
    "dow": "(int) day of week, 0..7",
    "day": "(int) day in month",
    "month": "(int) month number",
    "week number": "(int) iso week number (0..53)",
    "month week number": "(int) month week number (0..5)",
    "year": " (int) year",
    "public holidays": "(bool) Public holiday in France",
}

In [ ]:
cols = columns_descr.keys()
daily = daily.loc[:, columns_descr.keys()]
daily["public holidays"] = daily["public holidays"].apply(
    lambda x: 1 if x is True else 0
)
daily.head()

In [ ]:
plt.matshow(daily.corr())

There are highly correlate features, some may be discarded. Moreover, some features are redondant.

In [ ]:
del columns_descr["Chiffre d'Affaires HT"]

In [ ]:
# extract numeric features
num_cols = [
    col for col in columns_descr.keys() if np.isreal(daily.loc[start_date, col])
]
num_cols
# scatter_matrix(daily)

In [ ]:
scatter_matrix(daily.loc[:, num_cols], figsize=(15, 15))

 * The Income seems to vary each month
 * There are outliers in income
 * Mean product per transaction and mean transaction number is highly correlated (as expected) except for some days (must be treated separately)
 

In [ ]:
sns.jointplot(data=daily, x="Ticket moyen TTC", y="Nombre moyen de produits / Ticket")

* 3 days with higher mean transaction
* 1 day with the mean product price is higher than usual

### Transaction outliers

The goal of this section is to know if this outliers should be discarded.

In [ ]:
daily.loc[daily["Ticket moyen TTC"] > 60]

One of this 3 outlier correspond to a privatisation. This individual will be discarded.

In [ ]:
daily = daily.loc[daily["Ticket moyen TTC"] < 60]
# daily.head()

The opening day should alsobe discarded

In [ ]:
daily.loc["2018-09-08"]

In [ ]:
daily = daily[daily["date"] != "2018-09-08"]

###  Per  month income

In [ ]:
xlim = (0, max(daily["Chiffre d'Affaires TTC"]))
for year, month in product(range(2018, 2020), range(1, 13)):
    cur_data = daily.loc[(daily["month"] == month) & (daily["year"] == year)]
    if len(cur_data) == 0:
        continue
    plt.figure()
    # cur_data["Chiffre d'Affaires TTC"].plot(kind="box",)
    cur_data["Chiffre d'Affaires TTC"].plot(
        kind="hist", xlim=xlim, title="Income for {}/{}".format(month, year), bins=20
    )

For almost all month, there is one and only one day with high income

In [ ]:
daily[daily["Chiffre d'Affaires TTC"] > 1500]

Each month, one event help raising the income:
* 2018-09-08: opening
* 2018-10-31: halloween
* 2018-11-10: Nabla & JM
* 2018-12-13: Christmas co-plateau
* 2018-01-05: privatisation

In [ ]:
for title, group in daily.groupby("dow"):
    plt.figure()
    group["Chiffre d'Affaires TTC"].plot(kind="hist", title=title)

In [ ]:
t = daily.loc[:, ["dow", "Chiffre d'Affaires TTC"]]
t = t.groupby("dow").describe()

In [ ]:
daily.boxplot(column="Chiffre d'Affaires TTC", by="dow")
daily.boxplot(column="Chiffre d'Affaires TTC", by="month")

In [ ]:
t = daily.loc[:, ["month", "Chiffre d'Affaires TTC"]]
t = t.groupby("month").describe()

In [ ]:
t = daily.loc[:, ["day", "Chiffre d'Affaires TTC"]]
t = t.groupby("day").describe()

In [ ]:
daily["Chiffre d'Affaires TTC"].plot()

This graph denote weekly seasonalities

In [ ]:
t = daily.loc[:, ["month week number", "Chiffre d'Affaires TTC"]]
t = t.groupby("month week number").describe()
# t.plot(figsize=(15,7))
# t.iloc[:, [4, 5, 6]].plot(figsize=(15,7))
daily.boxplot(column="Chiffre d'Affaires TTC", by="month week number")

In [ ]:
t = daily.loc[:, ["month week number", "dow", "Chiffre d'Affaires TTC"]]
t = t.groupby(["month week number", "dow"]).describe()

# t.iloc[:, [4, 5, 6]].plot(figsize=(15,7))
daily.boxplot(
    column="Chiffre d'Affaires TTC", by=["month week number", "dow"], figsize=(15, 7)
)

The 1st friday after the 1st monday of each month is really a good day.

## Predictions

### time series analisis

In [ ]:
from datetime import timedelta
from numpy.linalg import LinAlgError
from sklearn.linear_model import LinearRegression

# from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.seasonal import seasonal_decompose

clean serie

In [ ]:
t = daily["Chiffre d'Affaires TTC"].resample("1D").mean()
t = t.interpolate()

zeros = pd.Series(
    0, index=[d for d in pd.date_range(start_date, end_date) if d not in open_days]
)
for day in zeros.index:
    t[day] = 0

cleaned_daily = t
cleaned_daily.describe()

In [ ]:
cleaned_daily.plot()

In [ ]:
split_date = pd.date_range(start_date, end_date, 5)[3]
daily_X_train_test = pd.DataFrame(cleaned_daily)
daily_X_train, daily_X_test = (
    daily_X_train_test[daily_X_train_test.index < split_date],
    daily_X_train_test[daily_X_train_test.index >= split_date],
)

detrend

In [ ]:
index = np.arange(len(daily_X_train)).reshape(-1, 1)
TREND_REG = LinearRegression().fit(index, daily_X_train)
trend = TREND_REG.predict(index)
trend

In [ ]:
daily_X_train["trend"] = trend
# daily_X_train["detrend"] = cleaned_daily - trend
daily_X_train.describe()

In [ ]:
daily_X_train.plot(figsize=(15, 7))

In [ ]:
# trend function for forecasts
def forecast_trend(day1):
    """ Return the trend part for the date day
    """
    day0 = daily.index[0]
    day0 = datetime(day0.year, day0.month, day0.day)
    day1 = datetime(day1.year, day1.month, day1.day)
    return TREND_REG.predict(np.array([(day1 - day0).days]).reshape(1, -1))[0]


forecast_trend(datetime.now().date())

seasonalities

In [ ]:
seasons = seasonal_decompose(daily_X_train)
seasonal, trend, residual = seasons.seasonal, seasons.trend, seasons.resid

In [ ]:
daily_X_train["auto trend"] = trend["Chiffre d'Affaires TTC"]
daily_X_train["auto seasonals"] = seasonal["Chiffre d'Affaires TTC"]
daily_X_train["auto residuals"] = residual["Chiffre d'Affaires TTC"]
daily_X_train.plot(figsize=(15, 7))

As expected, there is a weekly seasonality.

In [ ]:
ws = daily_X_train.head(7).copy()
ws["dow"] = list(map(lambda x: x.weekday(), ws.index))
WEEKLY_SEASON = dict(zip(ws["dow"], ws["auto seasonals"]))


def forecast_seasonality(day):
    """Return the seasonality component
    """
    return WEEKLY_SEASON[day.weekday()]


forecast_seasonality(datetime.now())

In [ ]:
trend_season = (
    daily_X_train.copy()
    .reset_index()["date"]
    .apply(lambda x: forecast_trend(x) + forecast_seasonality(x))
)

daily_X_train["trend + seasonal"] = list(trend_season)
daily_X_train.loc[:, ["Chiffre d'Affaires TTC", "trend + seasonal"]].plot(
    figsize=(15, 7)
)

residuals

In [ ]:
# daily_X_train["residual"] = daily_X_train["Chiffre d'Affaires TTC"] - daily_X_train["trend + seasonal"]
daily_X_train["Chiffre d'Affaires TTC"] - daily_X_train["trend + seasonal"]

# daily_X_train["residual"].plot(figsize=(15,7))

In [ ]:
res = {}
cleaned_ar = AR(cleaned_daily)

In [ ]:
res = cleaned_ar.fit()
# errors = pd.DataFrame(res).describe()
print(res)
res.k_ar
# res.predict(res.params, start=datetime.now())
daily_X_train.index[0], daily_X_train.index[-1]

### spectacle-based

In [ ]:
# TODO:
## data organisation
# each day:
# * fetch ordered list of {spectacle, duration, start_date, type}
# for each spectacle:
# * "chiffre d'affaire TTC"
# --------------------
# for each spectacle predict "chiffre d'affaire"

In [ ]:
import sklearn

In [ ]:
def extract_ca(d):
    cas = cleaned_daily[cleaned_daily.index == datetime(d.year, d.month, d.day)]
    if len(cas) > 0:
        return cas[0]
    return np.nan

In [ ]:
col = "Chiffre d'Affaires TTC"
events[col] = events["DATE"].apply(extract_ca)
events.head()

In [ ]:
events = events[events["TYPE"] != "ko"]

In [ ]:
events.boxplot("Chiffre d'Affaires TTC", by="TYPE", figsize=(15, 6), rot=45)

In [ ]:
events[events["TYPE"] == "concert"].boxplot(
    "Chiffre d'Affaires TTC", by="SUMMARY", figsize=(15, 6), rot=90
)

In [ ]:
events.boxplot("Chiffre d'Affaires TTC", by="DURATION", figsize=(15, 6), rot=45)

In [ ]:
events.boxplot("Chiffre d'Affaires TTC", by="START_TIME", figsize=(15, 6), rot=45)

In [ ]:
# feature selection
inf_cols = ["DURATION", "START_TIME", "TYPE", "Chiffre d'Affaires TTC"]

In [ ]:
# category encoding

categs = events["TYPE"].unique()
# encode_categ_index = pd.MultiIndex.from_tuples(product(["TYPE_ENCODE"], categs))
for categ in categs:
    colname = "TYPE_" + categ
    inf_cols.append(colname)
    events[colname] = events["TYPE"].apply(lambda x: 1 if x == categ else 0)
events.head()

In [ ]:
# transform to numerical
events["DURATION"] = events["DURATION"].apply(lambda x: x.total_seconds())
events["START_TIME"] = events["START_TIME"].apply(lambda x: x.hour * 60 + x.minute)

In [ ]:
event_X_train, event_X_test = (
    events[events["DATE"] < split_date.date()],
    events[events["DATE"] >= split_date.date()],
)
event_X_train = event_X_train[inf_cols]
event_X_train = event_X_train.dropna()

event_X_test = event_X_test[inf_cols]
event_X_test = event_X_test.dropna()

In [ ]:
event_X_train
event_Y_train = event_X_train["Chiffre d'Affaires TTC"]
event_Y_test = event_X_test["Chiffre d'Affaires TTC"]
cols = [col for col in inf_cols if col not in ("Chiffre d'Affaires TTC", "TYPE")]
event_X_train = event_X_train[cols]
event_X_test = event_X_test[cols]

event_X_train.head()

In [ ]:
impr_reg = sklearn.linear_model.LinearRegression()
impr_reg.fit(X=event_X_train, y=event_Y_train)

In [ ]:
results = pd.DataFrame(
    {"predict": impr_reg.predict(event_X_test), "real": event_Y_test}
)
results["error"] = abs(results["predict"] - results["real"])
results.head()